# Usare gli ambienti


Quando si esegue uno script come processo di Azure Machine Learning, è necessario definire il contesto di esecuzione per l'esecuzione del processo. Una configurazione chiave è la destinazione di calcolo in cui verrà eseguito lo script. Potrebbe trattarsi della workstation locale (in questo caso dell'istanza di calcolo) o di una destinazione di calcolo remota, ad esempio il cluster di calcolo gestito di Azure Machine Learning di cui è stato effettuato il provisioning su richiesta.

In questo notebook si creerà un cluster di calcolo ed si esploreranno le destinazioni di calcolo per i processi.

## Prima di iniziare

Per eseguire il codice in questo notebook, è necessaria la versione più recente del pacchetto  **azureml-ai-ml** . Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguire `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Dopo aver installato i pacchetti SDK necessari, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati automaticamente. Per completare il comando è necessario solo l'ID sottoscrizione.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro in alto a destra di Studio. Verrà aperto un riquadro a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Eseguire uno script come processo

Per eseguire il training di un modello, creare prima di tutto lo script **diabetes_training.py** nella cartella **src** . Lo script usa il file **diabetes.csv** nella stessa cartella dei dati di training.

Si noti che si importano librerie all'inizio dello script. Le funzioni di queste librerie vengono usate per elaborare i dati ed eseguire il training del modello. Indipendentemente dal calcolo usato per eseguire lo script, è necessario che queste librerie siano installate.

In [ ]:
Dopo aver creato lo script, è possibile eseguire lo script come processo. Lo script usa librerie comuni. È quindi possibile usare un ambiente curato che include pandas, numpy e scikit-learn, tra gli altri.

Il processo usa la versione più recente dell'ambiente curato: `AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`.

In [ ]:
Durante l'esecuzione del processo, è già possibile eseguire le celle successive.

## Elencare gli ambienti

Verranno ora esaminati gli ambienti all'interno dell'area di lavoro. 

Nel processo precedente è stato usato uno degli ambienti curati. Per esplorare tutti gli ambienti già esistenti nell'area di lavoro, è possibile elencare gli ambienti: 

In [ ]:
Si noti che tutti gli ambienti curati hanno nomi che iniziano **AzureML-** (non è possibile usare questo prefisso per i propri ambienti).

Per esaminare un ambiente specifico, è possibile recuperare un ambiente in base al nome e alla versione. Ad esempio, è possibile recuperare la *descrizione* e *i tag* dell'ambiente curato usato per il processo precedente:

## Creare e usare un ambiente personalizzato

Se un ambiente curato non include tutti i pacchetti Python necessari per eseguire lo script, è possibile creare un ambiente personalizzato. Elencando tutti i pacchetti necessari in un ambiente, è possibile eseguire di nuovo facilmente gli script. Tutte le dipendenze vengono archiviate nell'ambiente che è quindi possibile specificare nella configurazione del processo, indipendentemente dal calcolo usato.

Ad esempio, è possibile creare un ambiente semplicemente da un'immagine Docker. Alcuni framework come PyTorch avranno un'immagine Docker pubblica che include già tutti gli elementi necessari. 

Verrà ora creato un ambiente da un'immagine Docker:

In [ ]:
L'ambiente è ora registrato nell'area di lavoro ed è possibile farvi riferimento quando si esegue uno script come processo:

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Il processo avrà esito negativo rapidamente. Esaminare il messaggio di errore. </p>

Il messaggio di errore indica che non è presente alcun modulo denominato pandas. Esistono due possibili cause per un errore di questo tipo:

- Lo script usa pandas ma non importa la libreria (`import pandas as pd`). 
- Lo script importa la libreria nella parte superiore dello script, ma il calcolo non ha installato la libreria (`pip install pandas`).

Dopo aver esaminato lo `diabetes-training.py` script, è possibile osservare che lo script è corretto, il che significa che la libreria non è stata installata. In altre parole, l'ambiente non includeva i pacchetti necessari.

Verrà ora creato un nuovo ambiente usando l'immagine Docker di base usata nel processo precedente. A questo punto, si aggiungerà una specifica conda per assicurarsi che i pacchetti necessari vengano installati. Eseguire prima di tutto la cella seguente per creare il file di specifica conda:

Si noti che tutte le dipendenze necessarie sono incluse nel file di specifica conda per l'esecuzione corretta dello script.

Creare un nuovo ambiente usando l'immagine Docker di base **e** il file di specifica conda per aggiungere le dipendenze necessarie. Azure Machine Learning creerà l'ambiente conda sopra l'immagine Docker fornita. 

In [ ]:
È ora possibile inviare un processo con il nuovo ambiente per eseguire lo script:

L'invio del processo con il nuovo ambiente personalizzato attiva la compilazione dell'ambiente. La prima volta che si usa un ambiente appena creato, la compilazione dell'ambiente può richiedere da 10 a 15 minuti, il che significa anche che il processo richiederà più tempo. 

È anche possibile scegliere di attivare manualmente la compilazione dell'ambiente prima di inviare un processo. L'ambiente deve essere compilato solo la prima volta che viene usato. 

Quando il processo attiva la compilazione di un nuovo ambiente, è possibile esaminare i log della compilazione nella scheda **Output e log** del processo. Aprire **azureml-logs/20_image_build_log.txt** per esaminare i log della compilazione dell'ambiente. 

![Screenshot dei log di compilazione](./images/screenshot-logs.png)

In [ ]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

## Create and use a custom environment

If a curated environment doesn't include all the Python packages you need to run your script, you can create your own custom environment. By listing all necessary packages in an environment, you can easily re-run your scripts. All the dependencies are stored in the environment which you can then specify in the job configuration, independent of the compute you use.

For example, you can create an environment simply from a Docker image. Certain frameworks like PyTorch will have a public Docker image that already includes everything you need. 

Let's create an environment from a Docker image:

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> The job will quickly fail! Review the error message. </p>

The error message will tell you that there is no module named pandas. There are two possible causes for such an error:

- The script uses pandas but didn't import the library (`import pandas as pd`). 
- The script does import the library at the top of the script but the compute didn't have the library installed (`pip install pandas`).

After reviewing the `diabetes-training.py` script you can observe the script is correct, which means the library wasn't installed. In other words, the environment didn't include the necessary packages.

Let's create a new environment, using the base Docker image used in the previous job. Now, you'll add a conda specification to ensure the necessary packages will be installed. First, run the following cell to create the conda specification file:

In [ ]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.7
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Note that all necessary dependencies are included in the conda specification file for the script to run successfully.

Create a new environment using the base Docker image **and** the conda specification file to add the necessary dependencies. Azure Machine Learning will build the conda environment on top of the Docker image you provided. 

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Now, you can submit a job with the new environment to run the script:

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Submitting the job with the new custom environment triggers the build of the environment. The first time you use a newly created environment, it can take 10-15 minutes to build the environment, which also means your job will take longer to complete. 

You can also choose to manually trigger the build of the environment before you submit a job. The environment only needs to be built the first time you use it. 

When the job triggers the build of a new environment, you can review the logs of the build in the **Outputs + logs** tab of the job. Open **azureml-logs/20_image_build_log.txt** to inspect the logs of the environment build. 

![Screenshot build logs](./images/screenshot-logs.png)